## HW 4
Botasheva Zhanna

In [1]:
%matplotlib inline
import matplotlib
import random
#import wget
import os
import pathlib
import torch
import datetime
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import HTML
from pathlib import Path
from enum import Enum
from tqdm import tqdm
from functools import partial
from shutil import rmtree
#from razdel import tokenize
#from datasets import Dataset

from sklearn.metrics import (
    f1_score,
    matthews_corrcoef
)
from torch.optim import AdamW
from torch.utils.data import Dataset as TorchDataset
from torch.utils.data import (
    random_split,
    TensorDataset,
    DataLoader,
    RandomSampler,
    SequentialSampler
)
from transformers import (
    AutoTokenizer,
    BertTokenizer,
    T5Tokenizer,
    AutoModelForMaskedLM,
    BertForSequenceClassification,
    T5ForConditionalGeneration,
    AutoModel,
    AutoModelForCausalLM,
    get_linear_schedule_with_warmup,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

tqdm.pandas()

print('Import is done.')

Import is done.


## Конфигурация

In [2]:
class Configurator:
    def __init__(self) -> None:
        # Инициализация конфигурации с основными параметрами
        self._params = {
            'random': {'seed': 42},  # Задание числа для генерации случайных чисел

            'pretrained': {  # Параметры предобученных моделей
                'path': {
                    'bert': 'ai-forever/ruBert-base',  # Путь к модели BERT
                    'gpt': 'ai-forever/rugpt3large_based_on_gpt2',  # Путь к модели GPT
                    't5': 'ai-forever/ruT5-base',  # Путь к модели T5
                }
            },

            'optimizer': {  # Параметры для оптимизаторов
                'lr': {  # Скорости обучения
                    'bert': 2e-5,  # Скорость обучения для BERT
                    't5': 2e-5  # Скорость обучения для T5
                },
                'eps': {  # Маленькое значение для комфортного обновления весов
                    'bert': 1e-8,  # Эпсилон для BERT
                    't5': 1e-8  # Эпсилон для T5
                }
            },

            'dataset': {  # Параметры набора данных
                'name': {  # Имена наборов данных
                    'train': 'TRAIN',  # Имя обучающего набора
                    'test': 'TEST'  # Имя тестового набора
                },
            },

            'train': {  # Параметры обучения
                'size': 0.9,  # Размер обучающей выборки (90%)
                'epochs': {  # Количество эпох для обучения
                    'bert': 1,  # Эпохи для BERT
                    't5': 2  # Эпохи для T5
                }
            },

            'batch-size': 32  # Размер батча
        }

    def __call__(self, *args, **kwargs):
        # Позволяет вызвать экземпляр Configurator как функцию для получения параметров
        param_name = args[0]  # Название параметра из аргументов
        path = param_name.split('.')  # Разделение имени параметра по точкам
        return self._get_next(path, self._params, param_name)  # Запускает поиск значения параметра

    def _get_next(self, path, params, param_name):
        result = None  # Инициализация результата
        if isinstance(params, dict) and len(path) > 0:  # Проверка на словарь и наличие путей
            key = path.pop(0)  # Извлекаем первый элемент из пути
            if len(path) == 0:  # Если путь завершён
                if key in params:  # Проверяем, есть ли ключ в параметрах
                    result = params[key]  # Сохраняем результат
            else:  # Если путь не завершен, продолжаем поиск
                return self._get_next(path, params[key], param_name)
        if result is None:  # Если результат не найден
            print(f'Bad param name: {param_name}')  # Вывод сообщения об ошибке
        return result  # Возвращаем найденное значение или None

conf = Configurator()  # Создание экземпляра класса Configurator

print('Configuration block is done.')  # Сообщение о завершении настройки

Configuration block is done.


## Set random seed

In [3]:
# Установка начального значения для генератора случайных чисел библиотеки random
random.seed(conf('random.seed'))  # Получаем значение seed из конфигурации и применяем его

# Установка начального значения для генератора случайных чисел библиотеки numpy
np.random.seed(conf('random.seed'))  # Получаем значение seed из конфигурации и применяем его

# Установка начального значения для генератора случайных чисел в библиотеке PyTorch для CPU
torch.manual_seed(conf('random.seed'))  # Получаем значение seed из конфигурации и применяем его

# Установка начального значения для генератора случайных чисел в библиотеке PyTorch для GPU
torch.cuda.manual_seed(conf('random.seed'))  # Получаем значение seed из конфигурации и применяем его

print('Random seed is set.')  # Сообщение о том, что начальное значение генератора случайных чисел установлено

Random seed is set.


## Инициализация словаря

In [4]:
# Инициализация словаря total_result для хранения результатов различных моделей
total_result = {
    'bert': None,  # Результат для модели BERT
    'gpt-zero-shot': None,  # Результат для GPT в режиме нулевой подстановки
    'gpt-few-shot-3': None,  # Результат для GPT в режиме few-shot с 3 примерами
    'gpt-few-shot-5': None,  # Результат для GPT в режиме few-shot с 5 примерами
    'gpt-few-shot-10': None,  # Результат для GPT в режиме few-shot с 10 примерами
    't5': None  # Результат для модели T5
}

print('Total result is initialized.')  # Сообщение о том, что словарь total_result был инициализирован


Total result is initialized.


## Определения устройства

In [5]:
# Определение устройства (GPU или CPU) в зависимости от доступности GPU
if torch.cuda.is_available():  # Проверяем, доступен ли GPU
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')  # Выводим количество доступных GPU
    print(f'We will use the GPU: {torch.cuda.get_device_name(0)}')  # Указываем имя первого доступного GPU
    device = torch.device('cuda')  # Устанавливаем устройство для вычислений на GPU
else:
    print('No GPU available, using the CPU instead.')  # Если GPU недоступен, выводим сообщение
    device = torch.device('cpu')  # Устанавливаем устройство для вычислений на CPU

print('Device is defined.')  # Сообщение о том, что устройство для вычислений было определено

No GPU available, using the CPU instead.
Device is defined.


## Создание модели, токенизатора, оптимизатора

In [6]:
# Создание токенизатора для модели BERT из предобученной версии
bert_tokenizer = BertTokenizer.from_pretrained(conf('pretrained.path.bert'))

# Создание модели BERT для классификации последовательностей
bert_model = BertForSequenceClassification.from_pretrained(
    conf('pretrained.path.bert'),  # Загрузка предобученной модели BERT
    num_labels=2,  # Установка количества меток классов (например, бинарная классификация)
    output_attentions=False,  # Не сохранять внимательные веса
    output_hidden_states=False  # Не сохранять скрытые состояния
)

# Создание оптимизатора AdamW для модели BERT
bert_optimizer = AdamW(bert_model.parameters(), lr=conf('optimizer.lr.bert'), eps=conf('optimizer.eps.bert'))

# Создание токенизатора для модели GPT из предобученной версии
gpt_tokenizer = AutoTokenizer.from_pretrained(conf('pretrained.path.gpt'))

# Создание модели GPT для генерации текста с использованием предобученной модели
gpt_model = AutoModelForCausalLM.from_pretrained(conf('pretrained.path.gpt'))

# Создание токенизатора для модели T5 из предобученной версии (ускоренная версия не используется)
t5_tokenizer = T5Tokenizer.from_pretrained(conf('pretrained.path.t5'), use_fast=False)

# Создание модели T5 для условной генерации текста из предобученной версии
t5_model = T5ForConditionalGeneration.from_pretrained(conf('pretrained.path.t5'))

# Создание оптимизатора AdamW для модели T5
t5_optimizer = AdamW(t5_model.parameters(), lr=conf('optimizer.lr.t5'), eps=conf('optimizer.eps.t5'))

print('Models, tokenizers, optimizers are created.')  # Сообщение о том, что модели, токенизаторы и оптимизаторы созданы

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

Models, tokenizers, optimizers are created.


## Загрузка наборов данных

In [7]:
from google.colab import drive

# Монтируем Google Диск
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
df_train = pd.read_csv('/content/drive/MyDrive/data/train_data.csv')
df_test = pd.read_csv('/content/drive/MyDrive/data/test_data.csv')
df_train = df_train.dropna(subset=['Poem'])
df=pd.concat([df_train,df_test],axis=0)
df

,Genre,Poem
1,Music,In the thick brushthey spend the...
2,Music,Storms are generous. ...
3,Music,—After Ana Mendieta Did you carry around the ...
4,Music,for Aja Sherrard at 20The portent may itself ...
5,Music,"for Bob Marley, Bavaria, November 1980 Here i..."
...,...,...
145,Environment,"To pick a tulip from the garden, the red one. ..."
146,Environment,We are as clouds that veil the midnight moon; ...
147,Environment,"When pulled, the spider web took another form...."
148,Environment,Whose woods these are I think I know. His hous...


In [9]:
# Загрузка обучающего датасета в DataFrame с выборкой определенных колонок из конфигурации
# train_eval_dataframe = pd.read_csv(conf('dataset.path.train'), usecols=conf('dataset.usecols'))
train_eval_dataframe = df_train.copy()
# Загрузка тестового датасета в DataFrame с выборкой определенных колонок из конфигурации
# test_dataframe = pd.read_csv(conf('dataset.path.test'), usecols=conf('dataset.usecols'))
test_dataframe = df_test.copy()

# Получение случайной выборки индексов для валидации на основе заданного размера и случайного зерна
random_index = train_eval_dataframe.sample(frac=conf('train.size'), random_state=conf('random.seed')).index

# Формирование валидационного DataFrame, исключая случайные индексы
eval_dataframe = train_eval_dataframe[~train_eval_dataframe.index.isin(random_index)]

# Формирование обучающего DataFrame, включая только случайные индексы
train_dataframe = train_eval_dataframe[train_eval_dataframe.index.isin(random_index)]

print('Dataframes are loaded.')  # Сообщение о завершении загрузки DataFrame

# Вывод размеров обучающего и валидационного DataFrame
print(f'Train dataframe size: {len(train_dataframe)}')  # Размер обучающего DataFrame
print(f'Evaluate dataframe size: {len(eval_dataframe)}')  # Размер валидационного DataFrame
print(f'Test dataframe size: {len(test_dataframe)}')  # Размер тестового DataFrame

Dataframes are loaded.
Train dataframe size: 753
Evaluate dataframe size: 84
Test dataframe size: 150


## Максимальная длина стихотворения


In [10]:
def define_raw_max_length_by_bert(poems, raw_max_length):
    # Функция для определения сырой максимальной длины предложений на основе токенов BERT
    for poem in poems:
        # Кодирование предложения в токены, включая специальные токены
        input_ids = bert_tokenizer.encode(poem, add_special_tokens=True)
        # Обновление сырой максимальной длины, если длина текущего предложения больше
        raw_max_length = max(raw_max_length, len(input_ids))
    return raw_max_length  # Возвращение максимальной длины

def define_max_length(raw_max_length, threshold):
    # Функция для определения максимальной длины с учетом порога
    return threshold if threshold >= raw_max_length else define_max_length(raw_max_length, threshold * 2)

# Определение сырой максимальной длины для обучающего набора предложений
raw_max_length = define_raw_max_length_by_bert(train_dataframe.Poem.values, 0)

# Определение сырой максимальной длины для валидационного набора предложений, обновляя максимальную длину
raw_max_length = define_raw_max_length_by_bert(eval_dataframe.Poem.values, raw_max_length)

# Определение сырой максимальной длины для тестового набора предложений, обновляя максимальную длину
raw_max_length = define_raw_max_length_by_bert(test_dataframe.Poem.values, raw_max_length)

# Определение максимальной длины с начальным порогом 1
max_length = define_max_length(raw_max_length, 1)

print('Maximum length of poem is defined.')  # Сообщение о том, что максимальная длина предложений определена
print(f'Raw max_length is {raw_max_length}')  # Вывод сырой максимальной длины
print(f'Binary-based max_length is {max_length}')  # Вывод окончательной максимальной длины

Maximum length of poem is defined.
Raw max_length is 187
Binary-based max_length is 256


## Создание datasets & dataloaders

In [11]:
from sklearn.preprocessing import LabelEncoder

class T5TestDataset(TorchDataset):
    def __init__(self, text, tokenizer, length, device):
        # Инициализация T5TestDataset с текстом, токенизатором, максимальной длиной и устройством
        self._text = text.reset_index(drop=True)  # Сброс индексов текста
        self._tokenizer = tokenizer  # Токенизатор
        self._length = length  # Максимальная длина
        self._device = device  # Устройство (CPU/GPU)

    def __len__(self):
        return self._text.shape[0]  # Количество элементов в датасете

    def __getitem__(self, item):
        output = self._tokenize(self._text[item])  # Получение токенизированного текста
        return {k: v.reshape(-1).to(self._device) for k, v in output.items()}  # Возврат токенов на выбранном устройстве

    def _tokenize(self, text):
        # Токенизация текста с возвратом тензоров
        return self._tokenizer(text,
                               return_tensors='pt',
                               padding='max_length',
                               truncation=True,
                               max_length=self._length)


class T5TrainDataset(TorchDataset):
    POS_LABEL = 'верно'  # Позитивная метка
    NEG_LABEL = 'неверно'  # Негативная метка

    def __init__(self, text, label, tokenizer, length, device):
        # Инициализация T5TrainDataset с текстом, метками, токенизатором, максимальной длиной и устройством
        self._text = text.reset_index(drop=True)  # Сброс индексов текста
        self._label = label.reset_index(drop=True)  # Сброс индексов меток
        self._tokenizer = tokenizer  # Токенизатор
        self._length = length  # Максимальная длина
        self._device = device  # Устройство

    def __len__(self):
        return self._label.shape[0]  # Количество элементов в датасете

    def __getitem__(self, item):
        output = self._tokenize(self._text[item], self._length)  # Токенизация текста
        output = {k: v.reshape(-1).to(self._device) for k, v in output.items()}  # Возврат токенов на устройстве

        # Определение метки (позитивная или негативная)
        label = self.POS_LABEL if self._label[item] == 1 else self.NEG_LABEL
        label = self._tokenize(label, length=2).input_ids.reshape(-1).to(self._device)  # Токенизация метки

        output.update({'labels': label})  # Добавление метки в выходной словарь
        return output

    def _tokenize(self, text, length):
        # Токенизация текста с возвратом тензоров
        return self._tokenizer(text,
                               return_tensors='pt',
                               padding='max_length',
                               truncation=True,
                               max_length=length)


def create_bert_dataset(poems, genres, max_length):
    # Функция для создания датасета для BERT
    input_ids = []
    attention_masks = []
    for poem in poems:
        # Кодирование предложений в идентификаторы и маски для внимания
        encoded_dict = bert_tokenizer.encode_plus(
            poem,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded_dict['input_ids'])  # Сохранение идентификаторов
        attention_masks.append(encoded_dict['attention_mask'])  # Сохранение масок внимания
    input_ids = torch.cat(input_ids, dim=0)  # Конкатенация идентификаторов
    attention_masks = torch.cat(attention_masks, dim=0)  # Конкатенация масок внимания

    # Кодирование меток с использованием LabelEncoder
    label_encoder = LabelEncoder()
    genres_encoded = label_encoder.fit_transform(genres)

    genres_tensor = torch.tensor(genres_encoded)  # Преобразование меток в тензор

    # Возврат TensorDataset для BERT
    return TensorDataset(input_ids, attention_masks, genres_tensor)

# Извлечение предложений и меток из обучающего, валидационного и тестового наборов данных
train_poem = train_dataframe.Poem.values
train_genre = train_dataframe.Genre.values
eval_poem = eval_dataframe.Poem.values
eval_genre = eval_dataframe.Genre.values
test_poem = test_dataframe.Poem.values
test_genre = test_dataframe.Genre.values

# Создание BERT датасетов
bert_train_dataset = create_bert_dataset(train_poem, train_genre, max_length)
bert_eval_dataset = create_bert_dataset(eval_poem, eval_genre, max_length)
bert_test_dataset = create_bert_dataset(test_poem, test_genre, max_length)

# Создание T5 датасетов
t5_train_dataset = T5TrainDataset(train_dataframe['Poem'], train_dataframe['Genre'], t5_tokenizer, max_length, device)
t5_eval_dataset = T5TrainDataset(eval_dataframe['Poem'], eval_dataframe['Genre'], t5_tokenizer, max_length, device)
t5_test_dataset = T5TestDataset(test_dataframe['Poem'], t5_tokenizer, max_length, device)

# Создание загрузчиков данных для BERT
bert_train_dataloader = DataLoader(
    bert_train_dataset,
    sampler=RandomSampler(bert_train_dataset),
    batch_size=conf('batch-size')  # Размер батча из конфигурации
)
bert_eval_dataloader = DataLoader(
    bert_eval_dataset,
    sampler=SequentialSampler(bert_eval_dataset),
    batch_size=conf('batch-size')  # Размер батча из конфигурации
)
bert_test_dataloader = DataLoader(
    bert_test_dataset,
    sampler=SequentialSampler(bert_test_dataset),
    batch_size=conf('batch-size')  # Размер батча из конфигурации
)

# Создание загрузчиков данных для T5
t5_train_dataloader = DataLoader(t5_train_dataset, batch_size=conf('batch-size'), shuffle=True)
t5_eval_dataloader = DataLoader(t5_eval_dataset, batch_size=conf('batch-size'))
t5_test_dataloader = DataLoader(t5_test_dataset, batch_size=conf('batch-size'))

print('BERT & T5 dataloaders are created.')  # Сообщение о том, что загрузчики данных для BERT и T5 созданы

BERT & T5 dataloaders are created.


## Создание scheduler

In [12]:
# Создание scheduler для оптимизатора BERT
bert_scheduler = get_linear_schedule_with_warmup(
    bert_optimizer,  # Оптимизатор, для которого создается планировщик
    num_warmup_steps=0,  # Количество шагов разминки (warmup)
    num_training_steps=len(bert_train_dataloader) * conf('train.epochs.bert')  # Общее количество шагов обучения
)

# Создание scheduler для оптимизатора T5
t5_scheduler = get_linear_schedule_with_warmup(
    t5_optimizer,  # Оптимизатор, для которого создается планировщик
    num_warmup_steps=0,  # Количество шагов разминки (warmup)
    num_training_steps=len(t5_train_dataloader) * conf('train.epochs.t5')  # Общее количество шагов обучения
)

print('BERT & T5 schedulers are created')  # Сообщение о том, что планировщики для BERT и T5 созданы

BERT & T5 schedulers are created


## BERT training

In [14]:
print(set(b_labels.cpu().numpy()))  # Проверка уникальных меток

NameError: name 'b_labels' is not defined

In [15]:
def create_bert_dataset(poems, genres, max_length):
    input_ids = []
    attention_masks = []
    for poem in poems:
        # Кодирование предложений в идентификаторы и маски для внимания
        encoded_dict = bert_tokenizer.encode_plus(
            poem,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    # Кодирование меток с использованием LabelEncoder
    label_encoder = LabelEncoder()
    genres_encoded = label_encoder.fit_transform(genres)

    # Проверка уникальности меток
    assert (genres_encoded >= 0).all()  # Метки не должны быть отрицательными
    unique_labels = np.unique(genres_encoded)
    print(f"Unique labels after encoding: {unique_labels}")  # Проверка уникальных меток
    genres_tensor = torch.tensor(genres_encoded)

    return TensorDataset(input_ids, attention_masks, genres_tensor)

In [41]:
device = torch.device('cuda')  # Пользуйтесь CPU вместо GPU

In [44]:
for step, batch in enumerate(bert_train_dataloader):
    b_input_ids, b_input_mask, b_labels = extract_from_batch(batch, device)

    # Проверка уникальных меток
    try:
        print(f"Unique labels before conversion: {set(b_labels.numpy())}")  # Проверка уникальных меток
        print(f"Shape of b_labels: {b_labels.shape}")  # Размер меток
    except Exception as e:
        print(f"Error while checking labels at step {step}: {e}")
        break  # Остановите выполнение, если произошла ошибка


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [16]:
def flat_accuracy_bert(predictions, labels):
    # Функция для вычисления точности предсказаний
    predictions_flat = np.argmax(predictions, axis=1).flatten()  # Получаем предсказанные классы
    labels_flat = labels.flatten()  # Выравниваем метки
    return np.sum(predictions_flat == labels_flat) / len(labels_flat)  # Вычисление точности

def format_time_bert(elapsed):
    # Функция для форматирования времени в удобный вид
    return str(datetime.timedelta(seconds=int(round(elapsed))))  # Конвертация времени в формат HH:MM:SS

def log_step(step: int, offset: int, length: int, t0):
    # Функция для логирования прогресса обучения
    if ((step + 1) % offset == 0 and not step == 0) or (step == length - 1):
        elapsed = format_time_bert(time.time() - t0)  # Время, прошедшее с начала
        print(f'\tBatch {step + 1} of {len(bert_train_dataloader)}, elapsed: {elapsed}')

def extract_from_batch(batch, device) -> tuple:
    # Функция для извлечения данных из батча и переноса их на устройство
    b_input_ids = batch[0].to(device)  # Идентификаторы токенов
    b_input_mask = batch[1].to(device)  # Маска внимания
    b_labels = batch[2].to(device)  # Метки
    return b_input_ids, b_input_mask, b_labels  # Возврат тензоров

bert_model.cuda()  # Перенос модели на GPU

training_stats = []  # Начальная инициализация статистики обучения
total_t0 = time.time()  # Время начала тренировки
train_dataloader_length = len(bert_train_dataloader)  # Длина загрузчика обучающего датасета
val_dataloader_length = len(bert_eval_dataloader)  # Длина загрузчика валидационного датасета

epochs = conf('train.epochs.bert')  # Получение количества эпох из конфигурации
for epoch_i in range(0, epochs):
    print(f'\n======= Epoch {epoch_i + 1} / {epochs} =======\n')
    print('Training...')

    t0 = time.time()  # Время начала текущей эпохи
    total_train_loss = 0  # Суммарная потеря за эпоху
    bert_model.train()  # Установка модели в режим обучения

    for step, batch in enumerate(bert_train_dataloader):
        b_input_ids, b_input_mask, b_labels = extract_from_batch(batch, device)  # Извлечение данных из батча
        bert_model.zero_grad()  # Обнуляем градиенты

        # Прямой проход через модель
        res = bert_model(
            b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask,
            labels=b_labels
        )
        loss = res['loss']  # Потеря на текущем батче

        total_train_loss += loss.item()  # Суммируем потери для подсчета средней
        loss.backward()  # Обратный проход для вычисления градиентов
        torch.nn.utils.clip_grad_norm_(bert_model.parameters(), 1.0)  # Обрезаем градиенты

        bert_optimizer.step()  # Обновление параметров оптимизатором
        bert_scheduler.step()  # Обновление скорости обучения

        log_step(step, 40, train_dataloader_length, t0)  # Логирование прогресса

    avg_train_loss = total_train_loss / train_dataloader_length  # Средняя потеря
    train_time = format_time_bert(time.time() - t0)  # Время, затраченное на обучение
    print(f'\n\tAverage training loss: {avg_train_loss}\n\tTraining epoch took: {train_time}')

    print('\nRunning validation...')
    t0 = time.time()  # Время начала валидации
    total_eval_accuracy = 0  # Счетчик общей точности оценки
    total_eval_loss = 0  # Счетчик общей потери оценки

    bert_model.eval()  # Установка модели в режим оценки

    for batch in bert_eval_dataloader:
        b_input_ids, b_input_mask, b_labels = extract_from_batch(batch, device)  # Извлечение батча

        with torch.no_grad():  # Отключение градиентов для валидации
            res = bert_model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask,
                labels=b_labels
            )
        loss = res['loss']  # Потеря на валидационном наборе
        logits = res['logits']  # Получение предсказаний

        total_eval_loss += loss.item()  # Суммируем потерю валидации

        logits = logits.detach().cpu().numpy()  # Переносим предсказания на CPU для дальнейшей обработки
        label_ids = b_labels.to('cpu').numpy()  # Переносим метки на CPU

        total_eval_accuracy += flat_accuracy_bert(logits, label_ids)  # Обновляем общую точность

    avg_val_accuracy = total_eval_accuracy / val_dataloader_length  # Средняя точность валидации
    avg_val_loss = total_eval_loss / val_dataloader_length  # Средняя потеря валидации
    val_time = format_time_bert(time.time() - t0)  # Время валидации
    print(f'\n\tAccuracy: {avg_val_accuracy}')
    print(f'\tValidation loss: {avg_val_loss}')
    print(f'\tValidation took: {val_time}')

print('\nTraining complete!')  # Сообщение о завершении обучения
print(f'Total training took: {format_time_bert(time.time() - total_t0)}')  # Общее время обучения

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

## BERT  testing

In [ ]:
bert_model.eval()  # Переключение модели BERT в режим оценки
predictions, true_labels = [], []  # Инициализация списков для предсказаний и истинных меток

# Проход по тестовому загрузчику
for batch in bert_test_dataloader:
    b_input_ids, b_input_mask, b_labels = extract_from_batch(batch, device)  # Извлечение данных из батча

    with torch.no_grad():  # Отключение градиентов для экономии памяти
        outputs = bert_model(
            b_input_ids,
            token_type_ids=None,  # Не используются токены типа
            attention_mask=b_input_mask  # Маска внимания
        )
    logits = outputs[0]  # Получение логитов из выхода модели
    logits = logits.detach().cpu().numpy()  # Перенос логитов на CPU и преобразование в numpy массив
    label_ids = b_labels.to('cpu').numpy()  # Перенос истинных меток на CPU и преобразование в numpy массив

    predictions.append(logits)  # Добавление логитов в список предсказаний
    true_labels.append(label_ids)  # Добавление истинных меток в список

# Конкатенация всех предсказаний и их преобразование в одномерный массив
flat_predictions = np.concatenate(predictions, axis=0)
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()  # Получение классов из логитов
flat_true_labels = np.concatenate(true_labels, axis=0)  # Конкатенация истинных меток

# Вычисление F1
f1 = f1_score(flat_true_labels, flat_predictions)
total_result['bert'] = f1  # Сохранение F1ы в результирующий словарь
print(f'BERT is tested, F1-score: {f1:.3f}')  # Вывод F1


BERT is tested, F1-score: 0.867


## GPT preparation

In [ ]:
def calc_gpt_loss(text):
    # Функция для вычисления потерь GPT модели для заданного текста
    inputs = gpt_tokenizer.encode(text, return_tensors='pt').reshape(-1).to(device)  # Токенизация текста
    with torch.no_grad():  # Отключение градиентов для экономии памяти
        loss = gpt_model(input_ids=inputs, labels=inputs).loss.item()  # Вычисление потерь модели
    return loss  # Возвращение потерь

def shot_gpt(begin: str, text: str, positive_statement: str, negative_statement: str):
    # Функция для сравнения потерь позитивного и негативного утверждений
    positive_loss = calc_gpt_loss(' '.join([begin, text, positive_statement]))  # Потеря для позитивного утверждения
    negative_loss = calc_gpt_loss(' '.join([begin, text, negative_statement]))  # Потеря для негативного утверждения

    # Возвращение 1, если потери для позитивного утверждения больше, иначе 0
    return 1 if positive_loss > negative_loss else 0

gpt_model.to(device)  # Перенос модели GPT на заданное устройство (CPU/GPU)

print('GPT preparation is done.')  # Сообщение о завершении подготовки модели GPT

GPT preparation is done.


## GPT zero shot

In [ ]:
# Определение задач для проверки текста на ошибки
tasks = [
    {
        'begin': 'Если ли здесь ошибка?',
        'positive_statement': 'Нет.',
        'negative_statement': 'Есть.'
    },
    {
        'begin': 'Если ли здесь ошибка?',
        'positive_statement': 'Отсутствует.',
        'negative_statement': 'Присутствует.'
    },
    {
        'begin': 'Если ли здесь ошибка?',
        'positive_statement': 'Предложение правильное.',
        'negative_statement': 'Допущена ошибка.'
    },
    {
        'begin': 'Проверь на ошибки.',
        'positive_statement': 'Нет',
        'negative_statement': 'Есть.'
    },
    {
        'begin': 'Проверь на ошибки.',
        'positive_statement': 'Отсутствуют.',
        'negative_statement': 'Присутствуют.'
    }
]

best_f1 = 0.0  # Инициализация переменной для хранения лучшей F1

# Цикл по задачам для проверки текста
for task in tasks:
    print(f'Begin: {task["begin"]}, positive_statement: {task["positive_statement"]}, negative_statement: {task["negative_statement"]}')

    # Определение функции для оценки текста с помощью моделирования zero-shot
    progress_function = lambda text: shot_gpt(task['begin'], text, task['positive_statement'], task['negative_statement'])

    # Применение функции к каждому предложению в тестовом наборе
    y_pred = test_dataframe['sentence'].progress_apply(progress_function)

    # Вычисление F1 на основе предсказаний и истинных меток
    f1 = f1_score(y_pred, test_dataframe["acceptable"])

    # Обновление лучшей F1
    best_f1 = max(best_f1, f1)

    print(f'F1-score: {f1}\n')  # Вывод F1 для текущей задачи

# Сохранение лучшей F1 в результатах
total_result['gpt-zero-shot'] = best_f1
print(f'\nGPT zero-shot is done, best F1-score: {best_f1:.3f}')  # Вывод результатов

Begin: Если ли здесь ошибка?, positive_statement: Нет., negative_statement: Есть.


100%|████████████████████████████████████████████████████████████████████████████████| 983/983 [00:45<00:00, 21.61it/s]


F1-score: 0.01349527665317139

Begin: Если ли здесь ошибка?, positive_statement: Отсутствует., negative_statement: Присутствует.


100%|████████████████████████████████████████████████████████████████████████████████| 983/983 [00:45<00:00, 21.60it/s]


F1-score: 0.13184079601990048

Begin: Если ли здесь ошибка?, positive_statement: Предложение правильное., negative_statement: Допущена ошибка.


...

F1-score: 0.8543123543123543

Begin: Проверь на ошибки., positive_statement: Нет, negative_statement: Есть.


100%|████████████████████████████████████████████████████████████████████████████████| 983/983 [00:45<00:00, 21.65it/s]


F1-score: 0.14390243902439026

Begin: Проверь на ошибки., positive_statement: Отсутствуют., negative_statement: Присутствуют.


100%|████████████████████████████████████████████████████████████████████████████████| 983/983 [00:45<00:00, 21.43it/s]

F1-score: 0.19437939110070257


GPT zero-shot is done, best F1-score: 0.854


## GPT few shot (3)

In [ ]:
promt = """Проверить корректность предложения:
Иван вчера не позвонил. => Верно
Лесные запахи набегали волнами; в них смешалось дыхание можжевельника, вереска, брусники. => Верно
У многих туристов, кто посещают Кемер весной, есть шанс застать снег на вершине горы Тахталы и даже сочетать пляжный отдых с горнолыжным. => Неверно
"""

# Применение функции для оценки текста на каждом предложении в test_dataframe
y_pred = test_dataframe['sentence'].progress_apply(lambda text: shot_gpt(promt, text, ' => Верно', ' => Неверно'))

# Вычисление F1 на основе предсказаний и истинных меток
f1 = f1_score(y_pred, test_dataframe["acceptable"])

# Сохранение F1 в результирующий словарь
total_result['gpt-few-shot-3'] = f1
print(f'GPT few-shot(3) is done, F1-score: {f1:.3f}')  # Вывод результатов


100%|████████████████████████████████████████████████████████████████████████████████| 983/983 [01:11<00:00, 13.84it/s]

GPT few-shot(3) is done, F1-score: 0.854


## GPT few shot (5)

In [ ]:
promt = """Проверить корректность предложения:
Вчера президент имел неофициальную беседу с английским послом. => Верно
А ты ехай прямо к директору театров, князю Гагарину. => Неверно
Коллега так и не признал вину за катастрофу перед коллективом. => Верно
Малыш уже уверенно читает слова через мягкий знак. => Неверно
Я говорил с ним только ради Вас. => Верно
"""

# Применение функции для оценки текста на каждом предложении в test_dataframe
y_pred = test_dataframe['sentence'].progress_apply(lambda text: shot_gpt(promt, text, ' => Верно', ' => Неверно'))

# Вычисление F1 на основе предсказаний и истинных меток
f1 = f1_score(y_pred, test_dataframe["acceptable"])

# Сохранение F1 в результирующий словарь
total_result['gpt-few-shot-5'] = f1
print(f'GPT few-shot(5) is done, F1-score: {f1:.3f}')  # Вывод результатов

100%|████████████████████████████████████████████████████████████████████████████████| 983/983 [01:11<00:00, 13.80it/s]

GPT few-shot(5) is done, F1-score: 0.854


## GPT few shot (10)

In [ ]:
promt = """Проверить корректность предложения:
Только бы он громко не закричал, когда найдет решение. => Верно
Но Коле не помог его иноверец. => Неверно
Гармоничные пропорции здания основаны на классических образцах. => Верно
Вчера мне нужно ехать на завод. => Неверно
Дело приняло дурной оборот. => Верно
Я стою поблизости от Центризбиркома и наблюдаю за тем, что происходит у входа. => Неверно
Вани не было в школе. => Верно
В качестве примеров были приведены случаи, о которых, кажется, что где-то я уже читал. => Неверно
Кожа у виска была желтой. => Верно
В эту минуту роман был прочитан. => Неверно
"""

# Применение функции для оценки текста на каждом предложении в test_dataframe
y_pred = test_dataframe['sentence'].progress_apply(lambda text: shot_gpt(promt, text, ' => Верно', ' => Неверно'))

# Вычисление F1 на основе предсказаний и истинных меток
f1 = f1_score(y_pred, test_dataframe["acceptable"])

# Сохранение F1 в результирующий словарь
total_result['gpt-few-shot-10'] = f1
print(f'GPT few-shot(10) is done, F1-score: {f1:.3f}')  # Вывод результатов


100%|████████████████████████████████████████████████████████████████████████████████| 983/983 [01:48<00:00,  9.05it/s]

GPT few-shot(10) is done, F1-score: 0.854


## T5 training

In [ ]:
t5_model.to(device)  # Перемещение модели T5 на заданное устройство (CPU или GPU)
n_epochs = conf('train.epochs.t5')  # Получение количества эпох из конфигурационного файла

dl_length = len(t5_train_dataloader)  # Длина обучающего загрузчика
for epoch in range(n_epochs):  # Цикл по количеству эпох
    print(f'\n======= Epoch {epoch + 1} / {n_epochs} =======\n')

    print('Training...\n')
    t5_model.train()  # Установка модели в режим обучения

    for batch_id, batch in enumerate(t5_train_dataloader):  # Итерация по батчам
        outputs = t5_model(**batch)  # Прямой проход через модель
        loss = outputs.loss  # Получение потерь

        loss.backward()  # Обратное распространение градиента
        t5_optimizer.step()  # Обновление параметров модели
        t5_scheduler.step()  # Обновление скорости обучения
        t5_optimizer.zero_grad()  # Обнуление градиентов для следующего батча

        # Логирование потерь каждые 10 батчей и на последнем батче
        if (((batch_id + 1) % 10 == 0) and not batch_id == 0) or (batch_id == dl_length - 1):
            print(f'\t\tBatch {batch_id + 1} of {dl_length}, loss : {loss.item():.3f}')

    print('Validation...')
    t5_model.eval()  # Установка модели в режим оценки

    with torch.no_grad():  # Отключение градиентов для экономии памяти
        eval_loss = [t5_model(**batch).loss.item() for batch in t5_eval_dataloader]  # Получение потерь для валидации

    print(f'\tValidation loss: {np.sum(eval_loss) / len(eval_loss)}')  # Вывод средней потери на валидации

print('Training complete!')  # Сообщение о завершении обучения


======= Epoch 1 / 2 =======

Training...

		Batch 10 of 222, loss : 5.728
		Batch 20 of 222, loss : 2.136
		Batch 30 of 222, loss : 1.363
		Batch 40 of 222, loss : 0.833
		Batch 50 of 222, loss : 0.582
		Batch 60 of 222, loss : 0.552
		Batch 70 of 222, loss : 0.529
		Batch 80 of 222, loss : 0.374
		Batch 90 of 222, loss : 0.644
		Batch 100 of 222, loss : 0.402
		Batch 110 of 222, loss : 0.395
		Batch 120 of 222, loss : 0.370
		Batch 130 of 222, loss : 0.294
		Batch 140 of 222, loss : 0.499
		Batch 150 of 222, loss : 0.300
		Batch 160 of 222, loss : 0.430
		Batch 170 of 222, loss : 0.437
		Batch 180 of 222, loss : 0.233
		Batch 190 of 222, loss : 0.542
		Batch 200 of 222, loss : 0.252
		Batch 210 of 222, loss : 0.401
		Batch 220 of 222, loss : 0.194
		Batch 222 of 222, loss : 0.191
Validation...
	Validation loss: 0.2917381364107132

======= Epoch 2 / 2 =======

Training...

		Batch 10 of 222, loss : 0.354
		Batch 20 of 222, loss : 0.500
		Batch 30 of 222, loss : 0.333
		Batch 40 of 222

## T5 testing

In [ ]:
# Токенизация положительного ярлыка
pos_label = t5_tokenizer(T5TrainDataset.POS_LABEL,
                         return_tensors='pt',
                         padding='max_length',
                         truncation=True,
                         max_length=2)['input_ids'][0][0].item()  # Получение токена для положительного ярлыка

t5_model.eval()  # Переключение модели T5 в режим оценки

result = np.array([])  # Инициализация массива для хранения результатов
for batch in t5_test_dataloader:  # Итерация по тестовому загрузчику
    tokens = t5_model.generate(**batch)  # Генерация токенов с помощью модели T5
    # Подсчет наличия положительного ярлыка в сгенерированных токенах
    tokens = [1 if pos_label in token else 0 for token in tokens]
    result = np.hstack([result, tokens])  # Объединение результатов с ранее полученными

# Вычисление F1 на основе предсказанных и истинных меток
f1 = f1_score(result, test_dataframe["acceptable"])
total_result['t5'] = f1  # Сохранение F1 в результирующий словарь

print(f'T5 testing is done, F1-score: {f1:.3f}')  # Вывод результатов

C:\Python310\lib\site-packages\transformers\generation\utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


T5 testing is done, F1-score: 0.854


## Результаты



In [ ]:
# Создание списка данных из словаря total_result
data = [[k, v] for k, v in total_result.items()]  # Генерация списка пар (тип модели, F1)

# Создание DataFrame на основе полученных данных
df = pd.DataFrame(data, columns=['Type', 'F1'])  # Указание названий столбцов

# Отображение DataFrame в HTML формате без индексов
HTML(df.to_html(index=False))  # Преобразование DataFrame в HTML и визуализация

Type,F1
bert,0.867310
gpt-zero-shot,0.854312
gpt-few-shot-3,0.854312
gpt-few-shot-5,0.854312
gpt-few-shot-10,0.854312
t5,0.854142


BERT является самой эффективной моделью для текущих задач обработки текста, в то время как различные варианты GPT демонстрируют схожую эффективность. Модель T5 выступает немного ниже, но также имеет неплохие результаты.